# ScanNet inference
This notebook lets you run inference on ScanNet Validation set, from a pretrained model's checkpoint on your machine. The output will be a confusion matrix which you can save locally on your machine. For this, you must have done the following:
- downloaded and preprocessed the ScanNet dataset (if you haven't, instantiating the dataset will launch it for you though)
- trained a model (which produced a checkpoint directory) or downloaded our pretrained weights

Note this notebook should work for multimodal (2D+3D) and unimodal (3D only) experiments.

In [3]:
# Select you GPU
I_GPU = 0

In [4]:
# Uncomment to use autoreload
# %load_ext autoreload
# %autoreload 2

import os
import sys
import numpy as np
import torch
import glob
from omegaconf import OmegaConf
import warnings
warnings.filterwarnings('ignore')

torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_points3d.trainer import Trainer

In [5]:
# Set your parameters
DATA_ROOT = '/mnt/hdd/datasets'
checkpoint_dir = '/home/rozenberszki/ADL4CV-MultiViewAggregation/DeepViewAgg/outputs/2023-01-03/10-29-23'
result_dir = 'result/test'
model_name = 'Res16UNet34-L4-early-ade20k-interpolate-local-fusion'                                   # adapt if you use another model in your checkpoint
split = 'val'                                                         # 'test' set will produce data for submission to the SCanNet 3D semantic segmentation benchmark
n_votes = 1                                                           # number of inferences per spherical sample. For multi-inference voting with inference-time augmentation
batch_size = 8                                                        # increase if your device allows it
full_res = True                                                       # predictions will be made on the raw point cloud, at full resolution
num_workers = 4                                                       # increase if your machine allows it
exp_name = None                                                       # you may give a name to the experiment
exp_name = f'{model_name}_{split}_votes-{n_votes}' if exp_name is None else exp_name

In [6]:
print(f'Inference on S3DIS exp={exp_name}')

# These are the arguments passed to Hydra. You could run the same thing 
# from CLI using `eval.py` with the command `python eval.py [...]`
overrides = [
    f'model_name={model_name}',
    f"checkpoint_dir={checkpoint_dir}",
    f'voting_runs={n_votes}',
    f'tracker_options.full_res={full_res}',
    'precompute_multi_scale=False',
    f'num_workers={num_workers}',
    f'batch_size={batch_size}',
    f'cuda={I_GPU}',
    'weight_name=latest',
    f'+data.dataroot={DATA_ROOT}',
]

# Parse the arguments with Hydra and OmegaConf
cfg = hydra_read(overrides, config_name='eval')
OmegaConf.set_struct(cfg, False)

# Create the Trainer instance from your checkpoint
trainer = Trainer(cfg)

# Update the val and test transforms to match train transforms for 
# inference-time augmentation
test_transform = trainer._dataset.test_dataset[0].transform
val_transform = trainer._dataset.val_dataset.transform

if trainer._model.is_multimodal:
    test_transform_image = trainer._dataset.test_dataset[0].transform_image
    val_transform_image = trainer._dataset.val_dataset.transform_image

    trainer._dataset.test_dataset[0].transform_image.transforms[2].use_coverage = False
    trainer._dataset.test_dataset[0].transform_image.transforms[2].credit = int(1920000 * 2)
    trainer._dataset.test_dataset[0].transform_image.transforms[3].sigma = 0.02
    trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.saturation = [0.8, 1.2]
    trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.brightness = [0.8, 1.2]
    trainer._dataset.test_dataset[0].transform_image.transforms[4].transform.contrast = [0.8, 1.2]

    trainer._dataset.val_dataset.transform_image.transforms[2].use_coverage = False
    trainer._dataset.val_dataset.transform_image.transforms[2].credit = int(1920000 * 2)
    trainer._dataset.val_dataset.transform_image.transforms[3].sigma = 0.02
    trainer._dataset.val_dataset.transform_image.transforms[4].transform.saturation = [0.8, 1.2]
    trainer._dataset.val_dataset.transform_image.transforms[4].transform.brightness = [0.8, 1.2]
    trainer._dataset.val_dataset.transform_image.transforms[4].transform.contrast = [0.8, 1.2]

# Run inference
trainer.eval(stage_name=split)
cm = trainer._tracker._full_confusion_matrix
torch.save(cm, f'{result_dir}/{exp_name}.pt')
if split != 'test':
    print(f'  mIoU={cm.get_average_intersection_union() * 100:0.2f}')
    print(f'  OA={cm.get_overall_accuracy() * 100:0.2f}')

Inference on S3DIS exp=Res16UNet34-L4-early-ade20k-interpolate-local-fusion_val_votes-1
backbone.down_modules.0.image.conv
backbone
head


  0%|          | 0/39 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 2947592) exited unexpectedly

Confusion matrices will be saved in your `result_dir`. Please refer to `torch_points3d/metrics/confusion_matrix` if you need to compute more metrics from the confusion matrices.